In [1]:
from pylsl import StreamInlet, resolve_stream
import numpy as np
import joblib  # Used for loading sklearn models
import sys
import os

sys.path.append('./src/processing')
from preprocessing import *



Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:

# Configuration
srate = 160  #Sampling rate of the EEG data, adjust as needed
epoch_length_sec = 5  # Length of the desired sample in seconds
samples_needed = srate * epoch_length_sec  # Number of samples needed for ~5 seconds
#manually from eegbci dataset
channel_names = ['FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 'CP5', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'CP6', 'Fp1', 'Fpz', 'Fp2', 'AF7', 'AF3', 'AFz', 'AF4', 'AF8', 'F7', 'F5', 'F3', 'F1', 'Fz', 'F2', 'F4', 'F6', 'F8', 'FT7', 'FT8', 'T7', 'T8', 'T9', 'T10', 'TP7', 'TP8', 'P7', 'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'P8', 'PO7', 'PO3', 'POz', 'PO4', 'PO8', 'O1', 'Oz', 'O2', 'Iz']

print("Looking for an EEG stream...")
streams = resolve_stream('type', 'EEG')

inlet = StreamInlet(streams[0])

def collect_and_save_single_sample(inlet, samples_needed):
    buffer = []  # Initialize the buffer to hold collected samples
    timestamps = []  # To store timestamps of each sample

    while len(buffer) < samples_needed:
        # Continuously pull samples
        sample, timestamp = inlet.pull_sample()
        if sample:
            buffer.append(sample)  # Add the sample to the buffer
            timestamps.append(timestamp)  # Add the timestamp

        if len(buffer) >= samples_needed:
            # Once we have enough samples, save and exit
            np.save('sample.npy', np.array(buffer))  # Save the buffer as a numpy file
            print(f"Saved ~{epoch_length_sec}-second sample with {len(buffer)} samples.")
            return  # Exit the function, effectively stopping data collection

# Call the function to collect, save, and then stop
collect_and_save_single_sample(inlet, samples_needed)


Looking for an EEG stream...


2024-03-11 16:19:59.280 (  16.543s) [          456F24]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2024-03-11 16:19:59.280 (  16.543s) [          456F24]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2024-03-11 16:19:59.281 (  16.543s) [          456F24]      netinterfaces.cpp:102   INFO| 	IPv4 addr: 7f000001
2024-03-11 16:19:59.281 (  16.543s) [          456F24]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2024-03-11 16:19:59.281 (  16.543s) [          456F24]      netinterfaces.cpp:105   INFO| 	IPv6 addr: ::1
2024-03-11 16:19:59.281 (  16.543s) [          456F24]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2024-03-11 16:19:59.281 (  16.543s) [          456F24]      netinterfaces.cpp:105   INFO| 	IPv6 addr: fe80::1%lo0
2024-03-11 16:19:59.281 (  16.543s) [          456F24]      netinterfaces.cpp:91    I

In [ ]:
sample = np.load('sample.npy')
sample.shape

In [ ]:
preprocessed_data = preprocess_single_trial(sample, srate, channel_names)

In [ ]:
# Configuration
#model_path = 'path/to/your/saved/model.joblib'  # Path to your trained model
srate = 160  # Sampling rate of the EEG data
epoch_length_sec = 5  # Length of an epoch in seconds
samples_per_epoch = srate * epoch_length_sec  # Number of samples per epoch

# Load your model
#model = joblib.load(model_path)

# Resolve the stream
print("Looking for an EEG stream...")
streams = resolve_stream('type', 'EEG', 'name', 'BioSemi')
inlet = StreamInlet(streams[0])

# Initialize a buffer for accumulating samples
buffer = []

def decode(epoch):
    """
    Process and decode an epoch of EEG data.
    """
    # Assuming you have defined preprocess_data and extract_features functions
    preprocessed_epoch = preprocess_data(epoch)  # Preprocess the epoch
    features = extract_features(preprocessed_epoch)  # Extract features
    prediction = model.predict([features])  # Sklearn expects a 2D array for the features
    return prediction

def preprocess_data(epoch):
    # Placeholder for your preprocessing steps
    return epoch  # Return the processed epoch

def extract_features(epoch):
    # Placeholder for your feature extraction logic
    return epoch.mean(axis=0) 

while True:
    # Pull sample from LSL stream
    sample, timestamp = inlet.pull_sample()
    buffer.append(sample)
    
    # Check if buffer has enough samples to form an epoch
    if len(buffer) >= samples_per_epoch:
        epoch = np.array(buffer[:samples_per_epoch])  
        buffer = buffer[samples_per_epoch:]  
        
        # Decode the epoch
        prediction = decode(epoch)
        print(f"Timestamp: {timestamp}, Prediction: {prediction}")
